In [3]:
import dask
import dask.array as da
from dask import do
from dask.multiprocessing import get
from time import time
import numpy as np
import metacsv
import csv
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import scipy.stats
import xarray as xr
import copy
import re


In [5]:
#read in csvv
def read(filename):
    with open(filename, 'rU') as fp:
        attrs, coords, variables = metacsv.read_header(fp, parse_vars=True)
        data = {'attrs': attrs, 'variables': variables, 'coords': coords}

        if 'csvv-version' in attrs:
            if attrs['csvv-version'] == 'girdin-2017-01-10':
                return read_girdin(data, fp)


def read_girdin(data, fp):
    reader = csv.reader(fp)
    variable_reading = None

    for row in reader:
        if len(row) == 0 or (len(row) == 1 and len(row[0].strip()) == 0):
            continue

        if row[0] in ['observations', 'prednames', 'covarnames', 'gamma', 'gammavcv', 'residvcv']:
            data[row[0]] = []
            variable_reading = row[0]
        else:
            if variable_reading is None:
                print "No variable queued."
                print row
            assert variable_reading is not None
            data[variable_reading].append(map(lambda x: x.strip(), row))

    data['observations'] = float(data['observations'][0][0])
    data['prednames'] = data['prednames'][0]
    data['covarnames'] = data['covarnames'][0]
    data['gamma'] = np.array(map(float, data['gamma'][0]))
    data['gammavcv'] = np.array(map(lambda row: map(float, row), data['gammavcv']))
    data['residvcv'] = np.array(map(lambda row: map(float, row), data['residvcv']))

    return data

def subset(csvv, prednames):
    toinclude = map(lambda predname: predname in prednames, csvv['prednames'])
    toinclude = np.where(toinclude)[0]

    subcsvv = copy.copy(csvv)
    subcsvv['prednames'] = [csvv['prednames'][ii] for ii in toinclude]
    subcsvv['covarnames'] = [csvv['covarnames'][ii] for ii in toinclude]
    subcsvv['gamma'] = csvv['gamma'][toinclude]
    if 'gammavcv' in csvv and csvv['gammavcv'] is not None:
        subcsvv['gammavcv'] = csvv['gammavcv'][toinclude, toinclude]

    return subcsvv

In [8]:
csvv = read('/Users/rhodiumgroup/data/gcp_stuff/MLE_splines_GMFD_03212017.csvv')
prednames = ['spline_variables-0','spline_variables-1','spline_variables-2','spline_variables-3','spline_variables-4','spline_variables-5','spline_variables-6','spline_variables-0','spline_variables-1','spline_variables-2','spline_variables-3','spline_variables-4','spline_variables-5','spline_variables-6','spline_variables-0','spline_variables-1','spline_variables-2','spline_variables-3','spline_variables-4','spline_variables-5','spline_variables-6']
subsetted = subset(csvv, prednames)
subsetted

{'attrs': Attributes
     oneline:        Mortality rate temperature MLE spline 8 knots GMFD 2 factor
     version:        Mortality_MLE_splines_GMFD2factor_2017_03_21
     dependencies:   GCP_Reanalysis\Mortality\MLE\data\mortality_AEA
     description:    MLE, GMFD2factor spline model, 8 knots -12,-7,0,10,18,2...
     csvv-version:   girdin-2017-01-10,
 'coords': <Empty Coordinates>,
 'covarnames': ['1',
  '1',
  '1',
  '1',
  '1',
  '1',
  '1',
  'climtas',
  'climtas',
  'climtas',
  'climtas',
  'climtas',
  'climtas',
  'climtas',
  'loggdppc',
  'loggdppc',
  'loggdppc',
  'loggdppc',
  'loggdppc',
  'loggdppc',
  'loggdppc'],
 'gamma': array([  4.44224278e-01,  -5.46965000e-04,   4.41760000e-04,
         -2.06169000e-04,   1.90692000e-04,   1.90872156e-01,
          9.18037326e+00,   1.06382992e-01,   6.64540470e-02,
          7.00468900e-02,   6.76762480e-02,   3.24889000e-02,
         -1.01764476e-01,  -1.60632614e-01,  -4.64026714e-01,
         -3.75326097e-01,  -3.50358610e

In [43]:
csvv_labor = read('/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_2factor_BEST_14feb.csvv')
prednames_labor = ['tasmax', 'tasmax2','tasmax3', 'tasmax4']
subsetted = subset(csvv_labor, prednames_labor)
#subsetted['covarnames']

In [44]:
#read in annual climate data
def do_climate_thing(path,climate_args, resample_time, resample_how):
    
    t1 = time()
    year = re.search(r'(\d{4})', path).group(0)
    regexed = pd.date_range(str(year), periods=365)
    
    ds = xr.open_dataset(path)
    ds['time'] = regexed
    ds = ds.resample(resample_time, 'time', how=resample_how)
    t2 = time()
    print("do_climate_thing {}".format(t2-t1))
    return ds[climate_args], year


In [45]:
def get_global_gamma_estimates(path_to_csvv, pvals,prednames):
    t1 = time()
    csvv = read(path_to_csvv)
    subsetted = subset(csvv, prednames)
    np.random.seed(pvals)
    p_adjusted_gammas = scipy.stats.multivariate_normal.rvs(subsetted['gamma'], subsetted['gammavcv'])
    t2 = time()
    print("get_global_gamma_estimates {}".format(t2-t1))
    return p_adjusted_gammas, pvals

In [46]:
def do_covariate_thing(global_gammas, covariates):
    '''
    compute the product of global gamma estimates and local covariates
    
    Returns
    -------
    Numpy Arrray: MxN matrix of IR x covariates  
    '''
    covs = covariates()
    t1 = time()
    local_covars = global_gammas*covs
    t2 = time()
    print("do_covariate_thing: {}".format(t2-t1))

    return local_covars
    
    
    

In [47]:
def stack_map_sum(IR_gammas, ds_resampled):
    #reshape so we can perform polynomial products
    t1 = time()
    #reshaped = IR_gammas.reshape(24378,4,4)
    t2 = time()
    print('reshape {}'.format(t2-t1))
    #take the product of each covariate and polynomial tasmax
    totals =  np.dot(IR_gammas.T, ds_resampled[0])
    #sum across the values and save to dictionary
    t3 = time()
    print('map totals {}'.format(t3-t2))
    summed = {i:x.sum() for i,x in enumerate(totals)}
    t4 = time()
    print('summed {}'.format(t2-t1))
    df = pd.DataFrame.from_dict(summed, orient='index')
    t5 = time()
    print('todf {}'.format(t5 - t4))
    year = pd.DatetimeIndex(ds_resampled.time.data).year[0]
    t6 = time()
    print("datetime: {}".format(t6-t5))
    return df, year

In [48]:
def xr_reshape(df,year):
    #reshape data into xarray dataset
    #with appropriate coords,dims, etc
    t1 = time()
    ds = xr.Dataset(df)
    ds.rename({0:'annual impacts', 'dim_0': 'IR_region'}, inplace=True)
    _year = pd.DatetimeIndex(start=str(year)+'-01-01', end=str(year)+'-12-31', freq='A')
    ds['time'] = _year
    t2 = time()
    print("xr_reshap: {}".format(t2-t1))

    return ds

In [49]:
def gen_covars():
    ones = np.ones((24378,4))
    covars = np.random.normal(500, 250, size=(24378,12))
    new = np.hstack((ones,covars))
    return new

In [50]:
def do_annual_impact(init_dict):
    t1 = time()
    ds_resampled, year = do_climate_thing(init_dict['climate_path'],init_dict['variable'], init_dict['resample_dim'], init_dict['resample_method'])
    global_gammas, _ = get_global_gamma_estimates(init_dict['csvv_path'], init_dict['pval'], init_dict['prednames'])
    IR_gammas = do_covariate_thing(global_gammas, init_dict['ir_covariates'])
    df, year = stack_map_sum(IR_gammas, ds_resampled)
    ds = xr_reshape(df, year)
    t2 = time()
    print('Total time for annual impact pipeline: {}'.format(t2 - t1))
    return ds

In [51]:
def do_run(list_of_runs, output_dir=None):
    '''
    Computes impacts by region for a series of years in a specification
    
    Parameters
    ----------
    list_of_runs: list of dicts
        each dict is a specification for a single year of impacts
    
    output_dir: str
        path where to save file
    '''
    ds_list = [do_annual_impact(spec) for spec in list_of_runs]
    ds = xr.auto_combine(ds_list) 
    return ds
        
    
    
    

In [52]:
from itertools import product, combinations
paths = glob.glob('/Users/rhodiumgroup/data/gcp_stuff/tasmax_day_aggregated_rcp45_r1i1p1_CCSM4_*.nc')
variable = ['tasmax']
csvv_path=['/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_2factor_BEST_14feb.csvv']
pval = [10]
prednames = [['tasmax','tasmax2', 'tasmax3', 'tasmax4']]
resample_dim = ['A']
resample_method = ['mean']
ir_covariates= [gen_covars]
 
s = product(paths, variable, csvv_path, pval,  ir_covariates, prednames, resample_dim, resample_method)
job_list = []
for item in s:
    job_spec = dict(climate_path=item[0], 
                variable=item[1],
                csvv_path=item[2], 
                pval=item[3],
                ir_covariates=item[4],
                prednames=item[5],
                resample_dim=item[6],
                resample_method=item[7]
                )
    job_list.append(job_spec)
    
#job_list   

In [130]:
#run = do_run(job_list[:2])

In [32]:
run.to_netcdf('/Users/rhodiumgroup/data/gcp_stuff/toy_impacts_4_19.nc')

In [33]:
xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/toy_impacts_4_19.nc')

<xarray.Dataset>
Dimensions:         (IR_region: 24378, time: 2)
Coordinates:
  * IR_region       (IR_region) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 ...
  * time            (time) datetime64[ns] 2006-12-31 2007-12-31
Data variables:
    annual impacts  (time, IR_region) float64 5.519e+04 1.064e+05 2.139e+05 ...

In [175]:
#I need this to load up a bu
init_dict = dict(climate_path='/Users/rhodiumgroup/data/gcp_stuff/tasmax_day_aggregated_rcp45_r1i1p1_CCSM4_2006.nc', 
                variable='tasmax',
                csvv_path='/Users/rhodiumgroup/data/gcp_stuff/labor_global_interaction_2factor_BEST_14feb.csvv', 
                pval=10,
                ir_covariates=new,
                prednames=['tasmax','tasmax2', 'tasmax3', 'tasmax4'],
                resample_dim='A',
                resample_method='mean'
                )


In [22]:
#the temp data for is reduced to this cubic spline thing
splined = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/tas_restrict_cubic_spline_aggregate_rcp85_r1i1p1_CCSM4.nc')
splined.spline_variables[0:3, 0:6, 0]

<xarray.DataArray 'spline_variables' (time: 3, nterms: 6)>
array([[  3.391314e+06,   1.803356e+06,   5.978129e+05,   4.909705e+04,
          6.909933e+02,   2.078389e-01],
       [  3.512287e+06,   1.848558e+06,   5.803855e+05,   3.597939e+04,
          1.172965e+02,   0.000000e+00],
       [  3.214724e+06,   1.644738e+06,   4.889150e+05,   2.451471e+04,
          3.441414e+01,   0.000000e+00]], dtype=float32)
Coordinates:
    SHAPENUM  int32 1
Dimensions without coordinates: time, nterms
Attributes:
    long_title: aggregation ofAnnually summed 6 terms obtained by restrict cubic spline on tasin impact regions
    units: degree celsius ^ 3
    source: Regional aggregated spline variables from /global/scratch/jiacany/nasa_bcsd/spline/tas/rcp85/CCSM4/tas_restrict_cubic_spline_rcp85_r1i1p1_CCSM4_2099.nc

In [55]:


var_tas_monthly = xr.open_dataset('/Users/rhodiumgroup/data/gcp_stuff/tasMonthly_PoPwt_aggregated_IR_rcp85_CCSM4_2016.nc')


In [23]:
#Find the mean ir level gdp between two time periods
#Find the mean global gdp between two time periods
social = pd.read_csv('/Users/rhodiumgroup/data/gcp_stuff/combined.csv',comment='#')
interval_mean_ir = social.loc[(social['year'] >= 2001) & (social['year'] <= 2010), ['hierid', 'value']].groupby(['hierid']).mean()
interval_mean_global = interval_mean_ir.mean()

/Users/rhodiumgroup/anaconda3/envs/geospatial/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#load the nightlight data. 
global_nightlights = pd.read_csv('/Users/rhodiumgroup/data/gcp_stuff/nightlight_weight.csv')

#Data cleaning
#temporarily set zero values to some value
global_nightlights.loc[global_nightlights['gdppc_ratio'] == 0.0, 'gdppc_ratio'] = 'hooyah'

#Set null or non-existent values to 1
global_nightlights.loc[global_nightlights['gdppc_ratio'] == '', 'gdppc_ratio'] = 1.

#Now take the global min from the values that have a value
glmin = global_nightlights['gdppc_ratio'].min()

#assign the min value to the places that were previously zero
global_nightlights.loc[global_nightlights['gdppc_ratio'] == 'hooyah', 'gdppc_ratio'] = glmin

#fill any nans with ones 
global_nightlights['gdppc_ratio'] = global_nightlights['gdppc_ratio'].fillna(value=1.)


NameError: name 'pd' is not defined

In [1]:
global_nightlights

NameError: name 'global_nightlights' is not defined

In [57]:
#read in gdp data
gdppc_merged = pd.read_csv('/Users/rhodiumgroup/data/gcp_stuff/gdppc-merged.csv',comment='#')

In [114]:
gdppc_merged.iloc[pd.isnull(gdppc_merged).any(1).nonzero()[0]]

baseline_means = gdppc_merged.loc[(gdppc_merged['year'] >= 2001) & (gdppc_merged['year'] <= 2010), ['model', 'scenario', 'hierid', 'value']].groupby(['hierid', 'scenario', 'model']).mean()
logged_baseline = np.log(baseline_means.value)
future_years = gdppc_merged.loc[(gdppc_merged['year'] > 2010)]

In [129]:
baseline_means.head()

#gdppc_merged.iloc[pd.isnull(gdppc_merged).any(1).nonzero()[0]]
#baseline_means


value
hierid scenario model             
ABW    SSP1     high   1339.270843
       SSP2     high   1339.270843
       SSP3     high   1339.270843
       SSP4     high   1339.270843
       SSP5     high   1339.270843

In [37]:
#merge the two datasets on hierid
merged = pd.merge(global_nightlights, gdppc_merged, on='hierid', how= 'inner')
#create new column called weights 
#the value in this column is the gdp weighted by some nightlight ratio
#We use the nightlight ratio because?
merged['weights'] = merged.gdppc_ratio* merged.value

#The output of this is nightlight weighted IR specific GDP values 


In [41]:
#merged[merged['iso'] == 'AFG']

In [302]:
#pd.DataFrame(df.values*df2.values, columns=df.columns, index=df.index)
#merged.loc[merged['hierid'] == 'AIA']

In [300]:
#returns the indexes of the 
#pd.isnull(global_nightlights).any(1).nonzero()[0]
#pd.isnull(df).any(1).nonzero()[0]
#There are many 
#pd.isnull(merged).any(1).nonzero()[0]
#global_nightlights.loc[global_nightlights['hierid'] == 'AIA']

,hierid,iso,gdppc_ratio
264,AIA,AIA,1.0


In [42]:
#takes some baseline and then separates the years as before and after baseline
#This gives a scenario specific baseline
weighted_scenariod_gdp_baselines = merged.loc[merged.year <= 2015].groupby(by=['scenario', 'hierid']).mean()

#We need to return this value by ssp and region for our baseline

In [50]:
popdens = pd.read_csv('/Users/rhodiumgroup/data/gcp_stuff/popop_baseline.csv')

In [46]:
weighted_scenariod_gdp_futures = merged.loc[merged.year > 2015].groupby(by=['scenario', 'hierid']).mean()


In [54]:
merged.head()

,hierid,iso,gdppc_ratio,year,model,scenario,value,weights
0,ABW,ABW,1.0,2000,high,SSP1,0.000000,0.000000
1,ABW,ABW,1.0,2005,high,SSP1,1267.326733,1267.326733
2,ABW,ABW,1.0,2010,high,SSP1,1411.214953,1411.214953
3,ABW,ABW,1.0,2015,high,SSP1,1737.288136,1737.288136
4,ABW,ABW,1.0,2020,high,SSP1,2145.161290,2145.161290


In [52]:
popdens.head()

,gadmid,hierid,color,ISO,popop
0,28115,CAN.1.2.28,1,CAN,606.338691
1,28116,CAN.1.17.403,2,CAN,102.876158
2,28119,CAN.2.34.951,3,CAN,10.799070
3,28120,CAN.11.259.4274,4,CAN,289.289784
4,28124,CAN.11.269.4448,5,CAN,0.384870
